# ACC

In [1]:
import os
import sys
import yaml
import argparse
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

### Load config

In [3]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

### Model and lead time to verify

In [4]:
model_name = 'Your_model'
lead_range = conf[model_name]['lead_range']
verif_lead_range = conf[model_name]['verif_lead_range']

leads_exist = list(np.arange(lead_range[0], lead_range[-1]+lead_range[0], lead_range[0]))
leads_verif = list(np.arange(verif_lead_range[0], verif_lead_range[-1]+verif_lead_range[0], verif_lead_range[0]))
ind_lead = vu.lead_to_index(leads_exist, leads_verif)

print('Verifying lead times: {}'.format(leads_verif))
print('Verifying lead indices: {}'.format(ind_lead))

Verifying lead times: [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
Verifying lead indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [5]:
verif_ind_start = 0; verif_ind_end = 2*(365+366+365)

path_verif = conf[model_name]['save_loc_verif']+'combined_acc_{}_{}_{}h_{}h_{}.nc'.format(
    verif_ind_start, verif_ind_end, verif_lead_range[0], verif_lead_range[-1], model_name)

### ACC compute

In [7]:
# ---------------------------------------------------------------------------------------- #
# temp function for lat / lon average
def sp_avg(DS, wlat):
    return DS.weighted(wlat).mean(['latitude', 'longitude'], skipna=False)

# ---------------------------------------------------------------------------------------- #
# ERA5 climatology info
ERA5_path_string = conf['ERA5_weatherbench']['save_loc_clim'] + 'ERA5_clim_1990_2019_6h_interp.nc'
ds_ERA5_clim = xr.open_dataset(ERA5_path_string)
# rename coordinate name to latitude, longitude
if 'lat' in ds_ERA5_clim.coords:
    ds_ERA5_clim = ds_ERA5_clim.rename({'lat': 'latitude'})
if 'lon' in ds_ERA5_clim.coords:
    ds_ERA5_clim = ds_ERA5_clim.rename({'lon': 'longitude'})

# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years
year_range = conf['ERA5_ours']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')
    
# Select the specified variables and their levels
variables_levels = conf['ERA5_ours']['verif_variables']

# subset merged ERA5
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)

# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*.nc'))

# pick years
year_range = conf[model_name]['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_OURS = [fn for fn in filename_OURS if any(year in fn for year in years_pick)]

L_max = len(filename_OURS)
print('Total number of initialization times: {}'.format(L_max))
assert verif_ind_end <= L_max, 'verified indices (days) exceeds the max index available'

filename_OURS = filename_OURS[verif_ind_start:verif_ind_end]

# ---------------------------------------------------------------------------------------- #
# latitude weighting
lat = xr.open_dataset(filename_OURS[0])["latitude"]
w_lat = np.cos(np.deg2rad(lat))
w_lat = w_lat / w_lat.mean()

# ---------------------------------------------------------------------------------------- #
# ACC compute

acc_results = []

for fn_ours in filename_OURS:
    
    # --------------------------------------------------------------- #
    # import and subset forecast
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variables_levels)
    ds_ours = ds_ours.isel(time=ind_lead)
    dayofyear = ds_ours['time.dayofyear']
    hourofday = ds_ours['time'].dt.hour
    ds_ours = ds_ours.load()
    
    # --------------------------------------------------------------- #
    # get ERA5 verification target
    ds_target = ds_ERA5_merge.sel(time=ds_ours['time']).load()
    
    # --------------------------------------------------------------- #
    # get ERA5 climatology
    ds_clim_target = ds_ERA5_clim.sel(dayofyear=dayofyear, hour=hourofday).load()
    
    # ========================================== #
    # ERA5 anomaly
    ds_anomaly_ERA5 = ds_target - ds_clim_target

    # fcst anomaly
    ds_anomaly_OURS = ds_ours - ds_clim_target
    
    # ========================================== #
    # anmalies --> ACC with latitude-based cosine weights (check sp_avg and w_lat)
    top = sp_avg(ds_anomaly_OURS*ds_anomaly_ERA5, w_lat)
    
    bottom = np.sqrt(
        sp_avg(ds_anomaly_OURS**2, w_lat) * sp_avg(ds_anomaly_ERA5**2, w_lat))
                
    acc_results.append((top/bottom).drop_vars('time'))
    
# Combine ACC results
ds_acc = xr.concat(acc_results, dim='days')

# Save
# print('Save to {}'.format(path_verif))
# # ds_acc.to_netcdf(path_verif)

Total number of initialization times: 2192


In [8]:
ds_acc['Z500']

<xarray.DataArray 'Z500' (days: 2, time: 40)>
array([[0.99888684, 0.99770491, 0.99722977, 0.99594201, 0.99482148,
        0.99289744, 0.99165091, 0.98974669, 0.98772674, 0.98556496,
        0.98317519, 0.97969095, 0.97645495, 0.97155054, 0.96630877,
        0.95812528, 0.94695093, 0.9333896 , 0.9185277 , 0.90669438,
        0.89021143, 0.87023358, 0.84576103, 0.82248702, 0.79510186,
        0.76904295, 0.74706081, 0.7358786 , 0.72507969, 0.71658277,
        0.70573561, 0.68959415, 0.66041593, 0.61546379, 0.56834285,
        0.53335016, 0.51363589, 0.50299361, 0.50073248, 0.49736216],
       [0.99914873, 0.99830281, 0.99748366, 0.99614341, 0.99528616,
        0.99381515, 0.99226153, 0.99054172, 0.98866265, 0.98622308,
        0.98349024, 0.9786803 , 0.973936  , 0.96739759, 0.95922436,
        0.94760205, 0.93246522, 0.91854233, 0.90317128, 0.88759829,
        0.87002007, 0.85274582, 0.83400347, 0.8187102 , 0.80454697,
        0.79121797, 0.77332374, 0.75391422, 0.73058732, 0.70883915,
        0.680512  , 0.6412091 , 0.59626876, 0.55361581, 0.51593511,
        0.47773698, 0.45218266, 0.43734064, 0.43261723, 0.42842584]])
Coordinates:
    dayofyear  (days, time) int64 1 1 1 2 2 2 2 3 3 ... 9 9 10 10 10 10 11 11 11
    hour       (days, time) int64 6 12 18 0 6 12 18 0 6 ... 18 0 6 12 18 0 6 12
Dimensions without coordinates: days, time